In [15]:
# import numpy as np
# import pandas as pd
# from sklearn.metrics.pairwise import euclidean_distances
# from sklearn.metrics.pairwise import linear_kernel
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.feature_extraction.text import TfidfVectorizer
# from collections import Counter
# import mysql.connector
# import warnings
# warnings.filterwarnings("ignore")



# # join and concatenate inputted columns
# def create_soup(x, features):
#     soup_parts = [str(x[feature]) for feature in features if x[feature] is not None]  # Convert to string and filter out None values
#     return ' '.join(soup_parts)

# # count number of likeable elements for film
# def count_likeable(row):
#     features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     atts = sum(1 for col in row[features] if col is not None)
#     cast = len(row['cast'].split(','))
#     return atts+cast

# # get user loved films from db
# def get_loved_films(user_id):

#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT tconst FROM user_loved_films WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     rows = mycursor.fetchall()

#     tconst_list = [tconst[0] for tconst in rows]

#     mycursor.close()
#     mydb.close()

#     loved_films_df = data[data['tconst'].isin(tconst_list)]

#     return loved_films_df

# # get user liked attributes from db
# def get_liked_attributes(user_id):

#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_liked_attributes WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     attribute_fetch = mycursor.fetchall()

#     tconst_list = [row[1] for row in attribute_fetch]
#     attribute_bin = [row[1:] for row in attribute_fetch]

#     mycursor.close()
#     mydb.close()

#     attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     liked_attributes_df = pd.DataFrame(columns=attributes_template)

#     x = 0
#     for tconst, att_bin in zip(tconst_list, attribute_bin):
#         row_values = [tconst]  # Initialize row values with tconst
#         for liked, attribute in zip(att_bin[1:], attributes_template[1:]):
#             film_att = data[data['tconst'] == tconst]

#             if liked == 1:
#                 row_values.append(film_att[attribute].values[0])
#             else:
#                 row_values.append(None) 
#         liked_attributes_df.loc[x] = row_values
#         x+=1

#     return liked_attributes_df

# # get user liked cast from db
# def get_liked_cast(user_id):
#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_liked_cast WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     cast_fetch = mycursor.fetchall()

#     mycursor.close()
#     mydb.close()

#     cast = [row[1:] for row in cast_fetch]
#     liked_cast_df = pd.DataFrame(cast, columns=['tconst', 'name'])

#     return liked_cast_df

# # get user watchlist from db
# def get_watchlist(user_id):
#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_watchlist WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))


#     watchlist_fetch = mycursor.fetchall()

#     mycursor.close()
#     mydb.close()

#     tconst_list = [row[1] for row in watchlist_fetch]
#     attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'cast', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     watchlist_df = pd.DataFrame(columns=attributes_template)

#     x = 0
#     for tconst in tconst_list:
#         watchlist_df.loc[x] = data[data['tconst'] == tconst][attributes_template].values[0]
#         x+=1

#     return watchlist_df

# # calculate % of liked attributes of film in profile
# def calculate_percentage(row):
#     total_attributes = row['total_likeable']
#     total_liked = row['num_liked_atts']
#     percentage = total_liked / total_attributes
#     return percentage 

# # count liked attributes for film
# def count_liked(row):
#     features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     atts = sum(1 for col in row[features] if pd.notna(col))  # Change this line
#     cast = 0
#     if not pd.isna(row['cast']):
#         cast = len(row['cast'].split(','))
    
#     return atts + cast

# # CREATE USER PROFILE
# def get_user_profile(user_id):
#     # get user film preferences from db
#     lovedFilms = get_loved_films(user_id)
#     likedAtt = get_liked_attributes(user_id)
#     likedCast = get_liked_cast(user_id)
#     watchlist = get_watchlist(user_id)

#     # merge liked cast with liked attribute table
#     likedCast_grouped = likedCast.groupby('tconst')['name'].apply(lambda x: ', '.join(x.dropna())).reset_index()
#     likedAttributes = pd.merge(likedAtt, likedCast_grouped, on='tconst', how='outer') #outer join instead of left?
#     likedAttributes.rename(columns={'name': 'cast'}, inplace=True)

#     # format user_profile dataframe
#     merged_love_liked = pd.concat([lovedFilms, likedAttributes], ignore_index=True)
#     user_profile = pd.merge(merged_love_liked, data[['tconst', 'total_likeable']], on='tconst', how='left')
#     user_profile = user_profile.drop(columns=['poster', 'total_likeable_x'])
#     user_profile.rename(columns={'total_likeable_y': 'total_likeable'}, inplace=True)

#     # initialise columns to calculate likeage
#     user_profile['num_liked_atts'] = 0
#     user_profile['likeage'] = 0.0
#     # count total liked attributes for each film in profile
#     for index, film in user_profile.iterrows():
#         tconst = film['tconst']
#         user_profile.loc[user_profile['tconst'] == tconst, 'num_liked_atts'] = count_liked(film)

#     # calculate likeage for each film in profile
#     user_profile['likeage'] = user_profile.apply(lambda row: calculate_percentage(row), axis=1)
#     user_profile.drop(columns=['num_liked_atts', 'total_likeable'], inplace=True)

#     return (user_profile, lovedFilms)

# # split user profile into likes by group
# def collate_liked_groups(user_profile):
#     group_dataframes = []

#     # Define dictionary to map group names to corresponding columns
#     group_columns = {
#         'liked_title_plot': ['primaryTitle', 'plot', 'likeage'],
#         'liked_cast': ['cast', 'likeage'],
#         'liked_crew': ['director', 'cinematographer', 'writer', 'producer', 'editor', 'composer', 'likeage'],
#         'liked_genre': ['genres', 'likeage'],
#         'liked_meta': ['averageRating', 'startYear', 'runtimeMinutes', 'likeage']
#     }

#     # Iterate over the dictionary and create group dataframes
#     for group_name, columns in group_columns.items():
#         group_df = user_profile[columns].copy()
#         # Drop rows with all NaN values in specified columns
#         group_df = group_df.dropna(subset=columns[:-1], how='all')
#         group_dataframes.append(group_df)

#     return group_dataframes

# # cosine similarity vector with tf-idf between films in row and column
# def create_similarity_vector(row, column):
#     tfidf = TfidfVectorizer(stop_words='english')
#     row_soup_temp = row.apply(lambda x: create_soup(x, row.columns), axis=1)
#     row_soup = row_soup_temp.fillna('')
#     row_matrix = tfidf.fit_transform(row_soup)

#     column_soup_temp = column.apply(lambda x: create_soup(x, column.columns), axis=1)
#     column_soup = column_soup_temp.fillna('')
#     column_matrix = tfidf.transform(column_soup)

#     return linear_kernel(row_matrix, column_matrix)

# # euclidean distance vector for numerical attributes of film
# def create_euclidean_vector(row, column):
#     scaler = MinMaxScaler()
#     row = row.fillna('')
#     column = column.fillna(0)

#     row_normalized = scaler.fit_transform(row)
#     column_normalized = scaler.transform(column)
#     distances = euclidean_distances(row_normalized, column_normalized)
#     euclidean_matrix = 1 / (1 + distances)
#     return euclidean_matrix

# # get top N films similar to user profile based on input vector (NOT CURRENTLY IN USE)
# def get_similar_films(vector, N):
#     filtered_vector = vector[~data['tconst'].isin(lovedFilms['tconst'])] #don't recommend film the user has already loved
#     mean_similarity = np.mean(filtered_vector, axis=1)
#     sorted_indices = np.argsort(mean_similarity)[::-1]
#     top_N = sorted_indices[:N]
#     return data.iloc[top_N]

# # function to calculate unified recommendations
# def get_unified_recommendations(user_profile_groups, similarity_vectors, N):

#     weighted_scores = {}
    
#     # scale similarity score in respective vector based on likeage of feature
#     for group, attributes in user_profile_groups.items():
#         similarity_vector = similarity_vectors[group][~data['tconst'].isin(user_profile['tconst'])] #don't recommend films already in profile
#         likeage_array = np.array(list(attributes['likeage'].tolist()))
#         weighted_similarity = similarity_vector * likeage_array

#         # meta columns have higher similarity since its numerical so adjust for this
#         if group == 'meta':
#             scaling_factor = 1 - weighted_similarity
#             weighted_similarity *= scaling_factor

#         # dictionary of groups and weighted similarity vectors 
#         weighted_scores[group] = weighted_similarity

#      # combine weighted similarity scores across all groups
#     combined_scores = np.sum(list(weighted_scores.values()), axis=0)

#     # calculate mean similarity scores
#     mean_similarity = np.mean(combined_scores, axis=1)

#     # sort the mean similarity scores and retrieve the top N indices
#     sorted_indices = np.argsort(mean_similarity)[::-1][:N]

#     return data.iloc[sorted_indices]


# data = pd.read_json('webpage/films.json')
# data['total_likeable'] = data.apply(lambda x: count_likeable(x), axis=1)
# data.head()


# # get update user profile and loved films
# get_profile = get_user_profile('57')
# user_profile = get_profile[0]
# lovedFilms = get_profile[1]

# # get groups of liked attributes
# grouped_likes = collate_liked_groups(user_profile)
# liked_plot = grouped_likes[0]
# liked_cast = grouped_likes[1]
# liked_crew = grouped_likes[2]
# liked_genre = grouped_likes[3]
# liked_meta = grouped_likes[4]

# # Define the list of features excluding 'likeage'
# plot_features = [col for col in liked_plot.columns if col != 'likeage']
# crew_features = [col for col in liked_crew.columns if col != 'likeage']
# cast_features = [col for col in liked_cast.columns if col != 'likeage']
# genre_features = [col for col in liked_genre.columns if col != 'likeage']
# meta_features = [col for col in liked_meta.columns if col != 'likeage']

# # Call the function with the selected columns
# plot_matrix = create_similarity_vector(data[plot_features], liked_plot[plot_features])
# crew_matrix = create_similarity_vector(data[crew_features], liked_crew[crew_features])
# cast_matrix = create_similarity_vector(data[cast_features], liked_cast[cast_features])
# genre_matrix = create_similarity_vector(data[genre_features], liked_genre[genre_features])
# meta_matrix = create_euclidean_vector(data[meta_features], liked_meta[meta_features])


# # Similarity vectors for each group
# similarity_vectors = {
#     'plot': plot_matrix,
#     'cast': cast_matrix,
#     'crew': crew_matrix,
#     'genre': genre_matrix,
#     'meta': meta_matrix
# }

# user_profile_groups = {
#     'plot': liked_plot,
#     'cast': liked_cast,
#     'crew': liked_crew,
#     'genre': liked_genre,
#     'meta': liked_meta
# }


# get_unified_recommendations(user_profile_groups, similarity_vectors, 20)



In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import mysql.connector
import warnings
warnings.filterwarnings("ignore")

data = pd.read_json('webpage/films.json')

# join and concatenate inputted columns
def create_soup(x, features):
    soup_parts = [str(x[feature]) for feature in features if x[feature] is not None]  # Convert to string and filter out None values
    return ' '.join(soup_parts)

# count number of likeable elements for film
def count_likeable(row):
    features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    atts = sum(1 for col in row[features] if col is not None)
    cast = len(row['cast'].split(','))
    return atts+cast

# get user loved films from db
def get_loved_films(user_id):

    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT tconst FROM user_loved_films WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    rows = mycursor.fetchall()

    tconst_list = [tconst[0] for tconst in rows]

    mycursor.close()
    mydb.close()

    loved_films_df = data[data['tconst'].isin(tconst_list)]

    return loved_films_df

# get user liked attributes from db
def get_liked_attributes(user_id):

    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_liked_attributes WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    attribute_fetch = mycursor.fetchall()

    tconst_list = [row[1] for row in attribute_fetch]
    attribute_bin = [row[1:] for row in attribute_fetch]

    mycursor.close()
    mydb.close()

    attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    liked_attributes_df = pd.DataFrame(columns=attributes_template)

    x = 0
    for tconst, att_bin in zip(tconst_list, attribute_bin):
        row_values = [tconst]  # Initialize row values with tconst
        for liked, attribute in zip(att_bin[1:], attributes_template[1:]):
            film_att = data[data['tconst'] == tconst]

            if liked == 1:
                row_values.append(film_att[attribute].values[0])
            else:
                row_values.append(None) 
        liked_attributes_df.loc[x] = row_values
        x+=1

    return liked_attributes_df

# get user liked cast from db
def get_liked_cast(user_id):
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_liked_cast WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    cast_fetch = mycursor.fetchall()

    mycursor.close()
    mydb.close()

    cast = [row[1:] for row in cast_fetch]
    liked_cast_df = pd.DataFrame(cast, columns=['tconst', 'name'])

    return liked_cast_df

# get user watchlist from db
def get_watchlist(user_id):
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_watchlist WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))


    watchlist_fetch = mycursor.fetchall()

    mycursor.close()
    mydb.close()

    tconst_list = [row[1] for row in watchlist_fetch]
    attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'cast', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    watchlist_df = pd.DataFrame(columns=attributes_template)

    x = 0
    for tconst in tconst_list:
        watchlist_df.loc[x] = data[data['tconst'] == tconst][attributes_template].values[0]
        x+=1

    return watchlist_df

# calculate % of liked attributes of film in profile
def calculate_percentage(row):
    total_attributes = row['total_likeable']
    total_liked = row['num_liked_atts']
    percentage = total_liked / total_attributes
    return percentage 

# count liked attributes for film
def count_liked(row):
    features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    atts = sum(1 for col in row[features] if pd.notna(col))  # Change this line
    cast = 0
    if not pd.isna(row['cast']):
        cast = len(row['cast'].split(','))
    
    return atts + cast

# CREATE USER PROFILE
def get_user_profile(user_id):
    # get user film preferences from db
    lovedFilms = get_loved_films(user_id)
    likedAtt = get_liked_attributes(user_id)
    likedCast = get_liked_cast(user_id)
    watchlist = get_watchlist(user_id)

    # merge liked cast with liked attribute table
    likedCast_grouped = likedCast.groupby('tconst')['name'].apply(lambda x: ', '.join(x.dropna())).reset_index()
    likedAttributes = pd.merge(likedAtt, likedCast_grouped, on='tconst', how='outer') #outer join instead of left?
    likedAttributes.rename(columns={'name': 'cast'}, inplace=True)

    # format user_profile dataframe
    merged_love_liked = pd.concat([lovedFilms, likedAttributes], ignore_index=True)
    user_profile = pd.merge(merged_love_liked, data[['tconst', 'total_likeable']], on='tconst', how='left')
    user_profile = user_profile.drop(columns=['poster', 'total_likeable_x'])
    user_profile.rename(columns={'total_likeable_y': 'total_likeable'}, inplace=True)

    # initialise columns to calculate likeage
    user_profile['num_liked_atts'] = 0
    user_profile['likeage'] = 0.0
    # count total liked attributes for each film in profile
    for index, film in user_profile.iterrows():
        tconst = film['tconst']
        user_profile.loc[user_profile['tconst'] == tconst, 'num_liked_atts'] = count_liked(film)

    # calculate likeage for each film in profile
    user_profile['likeage'] = user_profile.apply(lambda row: calculate_percentage(row), axis=1)
    user_profile.drop(columns=['num_liked_atts', 'total_likeable'], inplace=True)

    return (user_profile, lovedFilms)

# split user profile into likes by group
def collate_liked_groups(user_profile):
    group_dataframes = []

    # Define dictionary to map group names to corresponding columns
    group_columns = {
        'liked_title_plot': ['primaryTitle', 'plot', 'likeage'],
        'liked_cast': ['cast', 'likeage'],
        'liked_crew': ['director', 'cinematographer', 'writer', 'producer', 'editor', 'composer', 'likeage'],
        'liked_genre': ['genres', 'likeage'],
        'liked_meta': ['averageRating', 'startYear', 'runtimeMinutes', 'likeage']
    }

    # Iterate over the dictionary and create group dataframes
    for group_name, columns in group_columns.items():
        group_df = user_profile[columns].copy()
        # Drop rows with all NaN values in specified columns
        # group_df = group_df.dropna(subset=columns[:-1], how='all')
        group_dataframes.append(group_df)

    return group_dataframes

# cosine similarity vector with tf-idf between films in row and column
def create_similarity_vector(row, column):
    tfidf = TfidfVectorizer(stop_words='english')
    row_soup_temp = row.apply(lambda x: create_soup(x, row.columns), axis=1)
    row_soup = row_soup_temp.fillna('')
    row_matrix = tfidf.fit_transform(row_soup)

    column_soup_temp = column.apply(lambda x: create_soup(x, column.columns), axis=1)
    column_soup = column_soup_temp.fillna('')
    column_matrix = tfidf.transform(column_soup)

    return linear_kernel(row_matrix, column_matrix)

# euclidean distance vector for numerical attributes of film
def create_euclidean_vector(row, column):
    scaler = MinMaxScaler()
    row = row.fillna('')
    column = column.fillna(0)

    row_normalized = scaler.fit_transform(row)
    column_normalized = scaler.transform(column)
    distances = euclidean_distances(row_normalized, column_normalized)
    euclidean_matrix = 1 / (1 + distances)
    return euclidean_matrix

# # get top N films similar to user profile based on input vector (NOT CURRENTLY IN USE)
# def get_similar_films(vector, N):
#     # filtered_vector = vector[~data['tconst'].isin(lovedFilms['tconst'])] #don't recommend film the user has already loved
#     mean_similarity = np.mean(vector, axis=1)
#     sorted_indices = np.argsort(mean_similarity)[::-1]
#     top_N = sorted_indices[:N]
#     return data.iloc[top_N]


data['total_likeable'] = data.apply(lambda x: count_likeable(x), axis=1)


In [17]:
get_user_profile('57')[0]

,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,likeage
0,tt0298203,8 Mile,"For Jimmy Smith, Jr., life is a daily fight ju...",7.2,"Drama,Music",110,2002,"Eminem, Mekhi Phifer, Brittany Murphy, Kim Bas...",Curtis Hanson,Rodrigo Prieto,Scott Silver,"Brian Grazer, Jimmy Iovine",Craig Kitson,None,1.000000
1,tt0171359,Hamlet,Modern day adaptation of Shakespeare's immorta...,5.9,"Drama,Romance,Thriller",112,2000,"Ethan Hawke, Kyle MacLachlan, Sam Shepard, Dia...",Michael Almereyda,John de Borman,William Shakespeare,"Andrew Fierberg, Amy Hobby",None,Carter Burwell,1.000000
2,tt0944835,Salt,"As a CIA officer, Evelyn Salt swore an oath to...",6.4,"Action,Thriller",100,2010,"Liev Schreiber, Chiwetel Ejiofor, Daniel Olbry...",Phillip Noyce,Robert Elswit,Kurt Wimmer,"Lorenzo di Bonaventura, Sunil Perkash",None,James Newton Howard,1.000000
3,tt0116483,Happy Gilmore,Failed hockey player-turned-golf whiz Happy Gi...,7.0,"Comedy,Sport",92,1996,"Adam Sandler, Christopher McDonald, Julie Bowe...",Dennis Dugan,Arthur Albert,Tim Herlihy,Robert Simonds,Jeff Gourson,Mark Mothersbaugh,1.000000
4,tt0121765,Star Wars: Episode II - Attack of the Clones,Following an assassination attempt on Senator ...,6.6,"Action,Adventure,Fantasy",142,2002,"Hayden Christensen, Ewan McGregor, Christopher...",George Lucas,David Tattersall,Jonathan Hales,"Rick McCallum, Lorne Orleans",None,John Williams,1.000000
5,tt0105793,Wayne's World,The adventures of two amiably aimless metal-he...,7.0,"Comedy,Music",94,1992,"Mike Myers, Dana Carvey, Rob Lowe, Tia Carrere",Penelope Spheeris,Theo van de Sande,"Bonnie Turner, Terry Turner",Lorne Michaels,None,J. Peter Robinson,1.000000
6,tt0119698,Princess Mononoke,"Ashitaka, a prince of the disappearing Emishi ...",8.3,"Action,Adventure,Animation",134,1997,"Yôji Matsuda, Billy Crudup, Yuriko Ishida, Yûk...",Hayao Miyazaki,Atsushi Okui,Neil Gaiman,Toshio Suzuki,Takeshi Seyama,Joe Hisaishi,1.000000
7,tt2510894,Hotel Transylvania 2,When the old-old-old-fashioned vampire Vlad ar...,6.6,"Adventure,Animation,Comedy",89,2015,"Adam Sandler, Andy Samberg, Kevin James, Selen...",Genndy Tartakovsky,None,"Robert Smigel, Todd Durham",Michelle Murdocca,Catherine Apple,Mark Mothersbaugh,1.000000
8,tt17351924,Saltburn,Struggling to find his place at Oxford Univers...,7.0,"Comedy,Drama,Thriller",131,2023,"Barry Keoghan, Jacob Elordi, Richard E. Grant,...",Emerald Fennell,Linus Sandgren,None,"Josey McNamara, Margot Robbie",Victoria Boydell,Anthony Willis,1.000000
9,tt0175880,None,None,NaN,None,None,None,"Philip Seymour Hoffman, Tom Cruise",Paul Thomas Anderson,None,None,None,None,None,0.200000


In [20]:
# function to calculate unified recommendations
def get_unified_recommendations(user_profile_groups, similarity_vectors, exclude, N):

    weighted_scores = {}
    
    # scale similarity score in respective vector based on likeage of feature
    for group, attributes in user_profile_groups.items():
        similarity_vector = similarity_vectors[group] 
        likeage_array = np.array(list(attributes['likeage'].tolist()))
        weighted_similarity = similarity_vector * likeage_array
  
        # meta columns have higher similarity since its numerical so adjust for this
        if group == 'meta':
            scaling_factor = 1 - weighted_similarity
            weighted_similarity *= scaling_factor

        # dictionary of groups and weighted similarity vectors 
        weighted_scores[group] = weighted_similarity

     # combine weighted similarity scores across all groups
    combined_scores = np.sum(list(weighted_scores.values()), axis=0)

    # calculate mean similarity scores
    mean_similarity = np.mean(combined_scores, axis=1)

    sorted_indices = np.argsort(mean_similarity)[::-1]

    sorted_films = data.iloc[sorted_indices].copy()
    
    sorted_films['similarity'] = mean_similarity[sorted_indices]

    filtered_recommendations = sorted_films[~sorted_films['tconst'].isin(exclude['tconst'])]

    return filtered_recommendations[:N]


# get update user profile and loved films
get_profile = get_user_profile('57')
user_profile = get_profile[0]
lovedFilms = get_profile[1]

# get groups of liked attributes
grouped_likes = collate_liked_groups(user_profile)
liked_plot = grouped_likes[0]
liked_cast = grouped_likes[1]
liked_crew = grouped_likes[2]
liked_genre = grouped_likes[3]
liked_meta = grouped_likes[4]

# Define the list of features excluding 'likeage'
plot_features = [col for col in liked_plot.columns if col != 'likeage']
crew_features = [col for col in liked_crew.columns if col != 'likeage']
cast_features = [col for col in liked_cast.columns if col != 'likeage']
genre_features = [col for col in liked_genre.columns if col != 'likeage']
meta_features = [col for col in liked_meta.columns if col != 'likeage']

# Call the function with the selected columns
plot_matrix = create_similarity_vector(data[plot_features], liked_plot[plot_features])
crew_matrix = create_similarity_vector(data[crew_features], liked_crew[crew_features])
cast_matrix = create_similarity_vector(data[cast_features], liked_cast[cast_features])
genre_matrix = create_similarity_vector(data[genre_features], liked_genre[genre_features])
meta_matrix = create_euclidean_vector(data[meta_features], liked_meta[meta_features])

# Similarity vectors for each group
similarity_vectors = {
    'plot': plot_matrix,
    'cast': cast_matrix,
    'crew': crew_matrix,
    'genre': genre_matrix,
    'meta': meta_matrix
}

# meta_matrix
user_profile_groups = {
    'plot': liked_plot,
    'cast': liked_cast,
    'crew': liked_crew,
    'genre': liked_genre,
    'meta': liked_meta
}

# user_profile
get_unified_recommendations(user_profile_groups, similarity_vectors, lovedFilms, 10)


,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster,total_likeable,similarity
8817,tt5220122,Hotel Transylvania 3: Summer Vacation,"Dracula, Mavis, Johnny and the rest of the Dra...",6.3,"Adventure,Animation,Comedy",97,2018,"Adam Sandler, Andy Samberg, Kevin James, Selen...",Genndy Tartakovsky,None,Michael McCullers,Michelle Murdocca,Joyce Arrastia,"Mark Mothersbaugh, Tiësto",/gjAFM4xhA5vyLxxKMz38ujlUfDL.jpg,15,0.545559
13677,tt0108525,Wayne's World 2,A message from Jim Morrison in a dream prompts...,6.2,"Comedy,Music",95,1993,"Mike Myers, Dana Carvey, Christopher Walken, T...",Stephen Surjik,Francis Kenny,"Bonnie Turner, Terry Turner",Lorne Michaels,None,Carter Burwell,/xRzMaHvSJEBdqEVrscOnfyyCj2Y.jpg,15,0.533463
5513,tt0080872,Highpoint,James Hatcher embezzles ten million dollars fr...,4.6,"Action,Comedy,Thriller",88,1982,"Richard Harris, Christopher Plummer, Beverly D...",Peter Carter,Albert J. Dunk,"Richard A. Guttman, Ian Sutherland",Daniel M. Fine,None,Christopher Young,/ilQ8h7vkwHYemCFK508MCMsHC8j.jpg,15,0.484999
6975,tt13391202,Shark Huntress,"Shelia, an environmentalist venturing into the...",1.5,"Action,Adventure,Thriller",82,2021,"Dean Alexandrou, John Flano, Russell Geoffrey ...",John Riggins,None,"Dustin Alexander III, Dante Delmare",Randy Kalsi,Mark Deeter,Samuel Mizell,/nfzSSpQecKEZp0FSol14g4nqtTT.jpg,15,0.484019
15033,tt0758763,Deadly Lessons,"A mysterious college professor, Simon Conjurer...",2.7,"Action,Comedy,Thriller",138,2006,"Jon Voight, Marcus Shirock, Germaine De Leon, ...",Stuart Paul,Howard Atherton,Simon Paul,"Eric M. Breiman, Steven Paul",None,Michel Legrand,/3VNlYsFItz8PGVJirSTZuWzoSjj.jpg,15,0.476984
4999,tt14804080,Jungle Run,A brother and sister's journey into the jungle...,2.8,"Action,Adventure,Thriller",85,2021,"Richard Grieco, Wade Hunt Williams, Jack Pears...",Noah Luke,None,Marc Gottlieb,David Michael Latt,Rob Pallatina,Mikel Shane Prather,/n6jf8Un9VX3Sw8ZhQUFuFSG0nij.jpg,15,0.475010
6452,tt0116039,Day of the Warrior,The Legion to Ensure Total Harmony and Law (L....,3.8,"Action,Adventure,Thriller",96,1996,"Kevin Light, Cristian Letelier, Julie Strain, ...",Andy Sidaris,Mark Morris,None,Arlene Sidaris,Anthony Dalesandro,Ron Di Iulio,/zBuJFNpOMgEH0wUk0Qwas8heowI.jpg,15,0.471681
23218,tt0099978,Laser Mission,A CIA agent is sent to get Professor Braun bef...,3.5,"Action,Adventure,Thriller",84,1989,"Brandon Lee, Ernest Borgnine, Richard Cox, Deb...",BJ Davis,Hans Kühle Jr.,"Phillip Gutteridge, David A. Frank",None,E. Selavie,David Knopfler,/9DVTsh96dGeHJoaUCJd7H6cZE6y.jpg,15,0.470096
14594,tt0055916,Trial and Error,After nearly 40 years of waiting for his big c...,6.2,"Comedy,Drama,Thriller",88,1962,"Peter Sellers, Richard Attenborough, David Lod...",James Hill,Edward Scaife,"John Mortimer, Pierre Rouve",Dimitri De Grunwald,None,Ron Grainer,/uw8L3PweuqNFjjiIK4b9SXwWFe9.jpg,15,0.466194
2376,tt0047795,Abbott and Costello Meet the Mummy,"Stranded in Egypt, Bud and Lou find themselves...",6.2,"Action,Adventure,Comedy",79,1955,"Bud Abbott, Lou Costello, Michael Ansara, Mari...",Charles Lamont,George Robinson,"John Grant, Lee Loeb",Howard Christie,Russell F. Schoengarth,None,/aHKjPVnDwE6iiz3pVj7MGwXo8B.jpg,15,0.465165


In [23]:
# get top N films similar to user profile based on input vector (NOT CURRENTLY IN USE)
def get_similar_films(vector, exclude, N):
    mean_similarity = np.mean(vector, axis=1)
    sorted_indices = np.argsort(mean_similarity)[::-1]
    top_N = sorted_indices[:]
    sorted_films = data.iloc[top_N]
    sorted_films['similarity'] = mean_similarity[sorted_indices]

    filtered_recommendations = sorted_films[~sorted_films['tconst'].isin(exclude['tconst'])]

    return filtered_recommendations.iloc[:N]

get_similar_films(crew_matrix, lovedFilms, 5)

,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster,total_likeable,similarity
1111,tt2013293,The Wind Rises,A lifelong love of flight inspires Japanese av...,7.8,"Animation,Biography,Drama",126,2013,"Hideaki Anno, Hidetoshi Nishijima, Masahiko Ni...",Hayao Miyazaki,Atsushi Okui,None,"Toshio Suzuki, Geoffrey Wexler",Takeshi Seyama,Joe Hisaishi,/jfwSexzlIzaOgxP9A8bTA6t8YYb.jpg,15,0.076202
13677,tt0108525,Wayne's World 2,A message from Jim Morrison in a dream prompts...,6.2,"Comedy,Music",95,1993,"Mike Myers, Dana Carvey, Christopher Walken, T...",Stephen Surjik,Francis Kenny,"Bonnie Turner, Terry Turner",Lorne Michaels,None,Carter Burwell,/xRzMaHvSJEBdqEVrscOnfyyCj2Y.jpg,15,0.072902
20049,tt6587046,The Boy and the Heron,"While the Second World War rages, the teenage ...",7.6,"Adventure,Animation,Drama",124,2023,"Soma Santoki, Masaki Suda, Kô Shibasaki, Aimyon",Hayao Miyazaki,Atsushi Okui,None,Toshio Suzuki,"Takeshi Seyama, Rie Matsubara",Joe Hisaishi,/f4oZTcfGrVTXKTWg157AwikXqmP.jpg,15,0.071865
16890,tt0114694,Tommy Boy,"To save the family business, two ne’er-do-well...",7.1,"Adventure,Comedy",97,1995,"Chris Farley, David Spade, Brian Dennehy, Bo D...",Peter Segal,Victor J. Kemper,"Bonnie Turner, Terry Turner",Lorne Michaels,None,David Newman,/6m1xJqfViDkmNmKUKvTSJ5fo0k4.jpg,15,0.067778
8533,tt0120915,Star Wars: Episode I - The Phantom Menace,"Anakin Skywalker, a young slave strong with th...",6.5,"Action,Adventure,Fantasy",136,1999,"Ewan McGregor, Liam Neeson, Jake Lloyd, Natali...",George Lucas,David Tattersall,None,Rick McCallum,"Paul Martin Smith, Ben Burtt",John Williams,/6wkfovpn7Eq8dYNKaG5PY3q2oq6.jpg,15,0.062153


In [6]:
get_similar_films(cast_matrix, lovedFilms, 5)


,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster,total_likeable
21933,tt1010409,My DNA Says I Love You,"Gigi and Marlene, two single women both in the...",4.8,"Comedy,Romance,Sci-Fi",95,2007,"Peter Ho, Eddie Peng, Terri Kwan, Nan Yu",Yun-Chan Lee,Ting-chang Chin,Yao Meng,Michelle Yeh,Hsiao-Yun Ku,Cheng-Chun Liang,/efaRrqJLZCLZImgjRxEeBBnzJV6.jpg,16
4855,tt0064381,"Goodbye, Columbus","A Jewish man and a Jewish woman meet, and whil...",6.4,"Comedy,Drama,Romance",102,1969,"Richard Benjamin, Jack Klugman, Ali MacGraw, N...",Larry Peerce,Gerald Hirschfeld,"Philip Roth, Arnold Schulman",Stanley R. Jaffe,None,Charles Fox,/sbL9RzE4Udc7HFYWvKDPKedAqkT.jpg,15
1492,tt1456660,No Man's Land,"Pan Xiao, a young lawyer, goes to a rural smal...",7.3,"Adventure,Crime,Drama",118,2013,"Zheng Xu, Duobujie, Huang Bo, Nan Yu",Hao Ning,Jie Du,"Ping Shu, Aina Xing",None,Yuan Du,Nathan Wang,/k2Lbohi5cLDaYWgYgKME8HQDR1m.jpg,15
1318,tt5482570,Lord of Shanghai,A lowly brothel maid captures the attention of...,5.6,Drama,111,2016,"Jun Hu, Hao Qin, Rhydian Vaughan, Nan Yu",Sherwood Hu,Andrzej Sekula,"Pang Bei, Hong Ying",Peggy Chiao,None,Johnny Klimek,/kfPb6rgCMIy3ohZzUlwO9bQFudn.jpg,15
4088,tt0078044,The Other Side of the Mountain: Part II,The Other Side of the Mountain Part 2 is a 197...,6.2,"Biography,Drama,Romance",100,1978,"Timothy Bottoms, Marilyn Hassett, Nan Martin, ...",Larry Peerce,Ric Waite,Douglas Day Stewart,Edward S. Feldman,Walter Hannemann,Lee Holdridge,/6xCf2QT7YH1VfKtAXmvrt3IBpDg.jpg,16


In [21]:
get_similar_films(crew_matrix, 5)


,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster,total_likeable
18620,tt0051600,Fanfare,"In the small Dutch village of Lagerheide, two ...",7.3,"Comedy,Music",86,1958,"Hans Kaart, Bernard Droog, Wim Van den Heuvel,...",Bert Haanstra,Eduard van der Enden,Jan Blokker,Rudolf Meyer,Ralph Sheldon,Jan Mul,/5znrKi2sicXgr7RH51Unqk43zx1.jpg,16
248,tt8019694,Frankie,About three generations of a family grappling ...,5.4,"Drama,Romance",100,2019,"Ariyon Bakare, Isabelle Huppert, Sennia Nanua,...",Ira Sachs,Rui Poças,Mauricio Zacharias,"Saïd Ben Saïd, Michel Merkt",None,Dickon Hinchliffe,/sbfrPcVHhKU6sPUP7ZVPsy7li4o.jpg,15
6438,tt0105220,Decay,"1986, Chernobyl disaster. Couples, friends, an...",6.6,Drama,103,1990,"Sergey Shakurov, Stanislav Stankevich, Georgiy...",Mikhail Belikov,Vasiliy Trushkovskiy,Oleg Prihodko,Mikhail Kostyukovskiy,None,"Igor Stetsyuk, Vladimir Vysotskiy",/3Yq0CsEzbCCbk03V5fCsD0lW7vD.jpg,15
12947,tt0081496,Dolaon solimsa jubangjang,"Ko Lin kills the brother of Hung Lau, an outla...",5.7,"Action,Comedy",84,1980,"Jin-Hwa Jeong, Ryong Wang, Un Bi, Noo-lee Jin",Jeong-yong Kim,Il-man Jeong,Seok-hun Yun,Yong-deok Kim,Dong-Chun Hyeon,Cheol-hyeok Lee,/eiW7g32lk9lWokAEFSTO3jdOmg1.jpg,16
17484,tt0184526,Gorgeous,"When Ah Bu, a girl from a small fishing town i...",6.0,"Action,Comedy,Romance",121,1999,"Jackie Chan, Tony Leung Chiu-wai, Emil Chau, S...",Vincent Kok,Man-Po Cheung,"Ivy Ho, Yiu Fai Lo",Raymond Chow,None,Dan-yee Wong,/vzY0zAdfnLZaVQrQcivasBDTmm4.jpg,15


In [25]:
get_similar_films(genre_matrix, 5)


,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster,total_likeable
18650,tt23490532,Nachindi Girl Friendu,"Raja Ram aka Raja, a confident youngster who h...",5.3,"Action,Thriller",130,2022,"Sairaj Suthari, Srikanth Iyengar, Gayatri Bhar...",Guru Pawan,"Siddam Manohar, Siddam Naresh",None,Atluri Narayana Rao,Kala Sagar Udagandla,Gifton Elias,/kOyQqVenAMDjzXxOCYAVrMF4Cjh.jpg,15
17329,tt0419979,Alter Ego,Three aspiring young Japanese models are on th...,4.7,Horror,63,2002,"Hideo Sakaki, Nobuko Sakuma, Sena, Chieko Kawabe",Issei Shibata,Shin Hayasaka,None,Yukihiko Yamaguchi,Etsuko Kimura,Saku Sakamoto,/gum4xP07ZU1EkvpsExpCxisn44g.jpg,15
10041,tt11318086,The Fall of the Queens,"One mother, two sisters that make each other j...",5.2,Drama,83,2021,"Franco Rizzaro, Malena Filmus, Lola Abraldes, ...",Lucas Nazareno Turturro,Nicolas Trovato,"Constanza Boquet, Mauro Guevara",Victoria Aizenstat,None,Sebastián Escofet,/fv2z4hOFGQsZUci3NNGDwHKXxZH.jpg,15
1959,tt0420724,A Killer Without a Grave,Nakata (Keiichiro Akagi) is a nihilistic young...,6.7,"Action,Crime",87,1961,"Gorô Mutsumi, Keiichirô Akagi, Reiko Sasamori,...",Yôichi Ushihara,Shinsaku Himeda,Toshio Matsuura,None,Masanori Tsujii,Taiichirô Kosugi,/uuIVJo1dRGa8TVOK9QDOwtwmidz.jpg,16
10836,tt0201046,Diamond Safari,"Raphaël, a former paratrooper, is unable to ad...",6.0,"Crime,Drama,Romance",90,1966,"Jean-Louis Trintignant, Horst Frank, Hellmut L...",Michel Drach,Andréas Winding,Albert Kantof,None,Geneviève Winding,Ward Swingle,/vbhYov4nrp5nrUkSrDSSxzF2kTU.jpg,15
